# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [71]:
# import libraries
import re
import pickle
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import nltk

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV



In [44]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/Lara/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Lara/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/Lara/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
# load data from database
engine = create_engine('sqlite:///DisasterDB.db')
df = pd.read_sql("SELECT * FROM DisasterData", engine)
X = df['message']
Y = df.iloc[:,4:]
# Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [38]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def tokenize(text):
   
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [39]:
pipeline =  Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),

        ('moc', MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1))
    ])




### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = .2)
pipeline.fit(X_train, y_train)


Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  low

In [37]:
X_train.shape

(20972,)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [26]:
y_pred = pipeline.predict(X_test)
y_pred

array([[1, 1, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [36]:
for c_test, c_pred, col_name in zip(y_test.to_numpy().T, y_pred.T, Y.columns):
    print(col_name)
    print(classification_report(c_test, c_pred))

related
              precision    recall  f1-score   support

           0       0.74      0.27      0.39      1261
           1       0.80      0.97      0.88      3942
           2       0.45      0.22      0.30        41

    accuracy                           0.79      5244
   macro avg       0.66      0.48      0.52      5244
weighted avg       0.78      0.79      0.75      5244

request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      4354
           1       0.91      0.44      0.59       890

    accuracy                           0.90      5244
   macro avg       0.91      0.72      0.77      5244
weighted avg       0.90      0.90      0.88      5244

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5220
           1       0.00      0.00      0.00        24

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      524

### 6. Improve your model
Use grid search to find better parameters. 

In [63]:
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'moc__estimator__n_estimators': [50, 100, 200]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1 , verbose = 10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [64]:
idx_sample = np.random.randint(0, X.shape[0], int(0.15 * X.shape[0]))
X_train, X_test, y_train, y_test = train_test_split(X.iloc[idx_sample], Y.iloc[idx_sample], test_size = .2) 
cv.fit(X_train, y_train)


Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   47.7s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 12.9min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 15.1min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 17.7min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 20.0min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 22.0min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed: 24

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('text_pipeline',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('vect',
                                                                                         CountVectorizer(analyzer='word',
                                                                                                         binary=False,
                                                                                                         decode_error='strict',
                                                                                                         dtype=<class 'numpy.int64'>,
               

In [66]:
cv.best_params_


{'features__text_pipeline__tfidf__use_idf': False,
 'features__text_pipeline__vect__max_df': 0.5,
 'features__text_pipeline__vect__max_features': 5000,
 'features__text_pipeline__vect__ngram_range': (1, 2),
 'moc__estimator__n_estimators': 100}

In [67]:
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X, Y, test_size = .2) 

pipeline_optim = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize, max_df = 0.5, max_features = 5000, ngram_range = (1,2))),
                ('tfidf', TfidfTransformer(use_idf = False))
            ]))
        ])),

        ('moc', MultiOutputClassifier(RandomForestClassifier(n_estimators=100), n_jobs=-1))
    ])


pipeline_optim.fit(X_train_all, y_train_all)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  low

In [69]:
y_pred_all = pipeline_optim.predict(X_test_all)


In [70]:

for c_test, c_pred, col_name in zip(y_test_all.to_numpy().T, y_pred_all.T, Y.columns):
    print(col_name)
    print(classification_report(c_test, c_pred))

related
              precision    recall  f1-score   support

           0       0.68      0.42      0.52      1202
           1       0.84      0.94      0.89      3999
           2       0.64      0.33      0.43        43

    accuracy                           0.82      5244
   macro avg       0.72      0.56      0.61      5244
weighted avg       0.80      0.82      0.80      5244

request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      4365
           1       0.80      0.50      0.62       879

    accuracy                           0.90      5244
   macro avg       0.85      0.74      0.78      5244
weighted avg       0.89      0.90      0.89      5244

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5221
           1       0.00      0.00      0.00        23

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      524

/opt/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.90      0.95      0.92      3801
           1       0.83      0.72      0.77      1443

    accuracy                           0.88      5244
   macro avg       0.87      0.83      0.85      5244
weighted avg       0.88      0.88      0.88      5244

floods
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      4833
           1       0.91      0.53      0.67       411

    accuracy                           0.96      5244
   macro avg       0.94      0.76      0.83      5244
weighted avg       0.96      0.96      0.95      5244

storm
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      4736
           1       0.78      0.61      0.68       508

    accuracy                           0.95      5244
   macro avg       0.87      0.79      0.83      5244
weighted avg       0.94      0.95      0.94      5244

fire
 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [72]:
filename = 'finalized_model.sav'
pickle.dump(pipeline_optim, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.